# Emulation running from notebook

In [1]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

## Katrina ADCIRC set-up

In [2]:
import matplotlib.pyplot as plt
from sithom.plot import plot_defaults

plot_defaults()

/Users/simon/new-orleans/env/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [3]:
# six degrees of freedom search
from src.models.emu6d import SixDOFSearch

In [4]:
# Two degrees of freedom search [bearing and position]
from src.models.emulation import EmulationSmash

In [7]:
## Let's see if the six degrees of freedom search works
tf  = SixDOFSearch(dryrun=True, path="Test", seed=0)
tf.run_initial(samples=200)
tf.setup_active()
tf.run_active(100)
tf.save_gp_all()

Calling {'angle': -29.249998927116394, 'speed': 3.125, 'point_east': 0.9794999957084655, 'rmax': 16644.999861717224, 'pc': 93700.00004768372, 'xn': 1.3955000042915344, 'vmax': 54.01667}
/Users/simon/new-orleans/data/Test/0
called fake func
Calling {'angle': 65.25000214576721, 'speed': 14.375, 'point_east': 0.8535000085830687, 'rmax': 36364.99834060669, 'pc': 96059.99994277954, 'xn': 1.284500002861023, 'vmax': 54.01667}
/Users/simon/new-orleans/data/Test/1
called fake func
Calling {'angle': -0.4499995708465576, 'speed': 5.734999924898148, 'point_east': 0.46649994850158694, 'rmax': 52315.00041484833, 'pc': 95219.99979019165, 'xn': 1.1164999842643737, 'vmax': 54.01667}
/Users/simon/new-orleans/data/Test/2
called fake func
Calling {'angle': 80.54999828338623, 'speed': 19.955000042915344, 'point_east': 0.7365000128746032, 'rmax': 18384.999930858612, 'pc': 96019.99998092651, 'xn': 1.0504999935626984, 'vmax': 54.01667}
/Users/simon/new-orleans/data/Test/3
called fake func
Calling {'angle': 55

 /Users/simon/new-orleans/env/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/simon/new-orleans/env/lib/python3.8/site-packages/GPy/kern/src/stationary.py:484: RuntimeWarning:overflow encountered in multiply
 /Users/simon/new-orleans/env/lib/python3.8/site-packages/GPy/kern/src/stationary.py:484: RuntimeWarning:invalid value encountered in multiply
 /Users/simon/new-orleans/env/lib/python3.8/site-packages/GPy/kern/src/stationary.py:487: RuntimeWarning:overflow encountered in multiply
 /Users/simon/new-orleans/env/lib/python3.8/site-packages/GPy/kern/src/stationary.py:487: RuntimeWarning:invalid value encountered in multiply
 /Users/simon/new-orleans/env/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Calling {'angle': -87.01368610603919, 'speed': 19.43477897361287, 'point_east': -0.4108169902424224, 'rmax': 53563.97382405253, 'pc': 97418.52341006456, 'xn': 0.8245738921756458, 'vmax': 54.01667}
/Users/simon/new-orleans/data/Test/200
called fake func
Calling {'angle': -84.25305614192669, 'speed': 19.521569357775345, 'point_east': -0.474398831886726, 'rmax': 56019.27417997999, 'pc': 91369.22375998426, 'xn': 0.8699636691368843, 'vmax': 54.01667}
/Users/simon/new-orleans/data/Test/201
called fake func
Calling {'angle': 75.08283984419381, 'speed': 19.110531491437833, 'point_east': 0.959738635593237, 'rmax': 2621.4251665410884, 'pc': 90907.37851293449, 'xn': 0.8282023075960084, 'vmax': 54.01667}
/Users/simon/new-orleans/data/Test/202
called fake func
Calling {'angle': -76.4507195903613, 'speed': 19.787797182247854, 'point_east': 1.1371836494478118, 'rmax': 12186.107971397747, 'pc': 90030.68617336122, 'xn': 0.9207233125849424, 'vmax': 54.01667}
/Users/simon/new-orleans/data/Test/203
called